# Dandiset 001275 Explorer Notebook

**Important:** This notebook was automatically generated using *dandi-notebook-gen* and has not been fully reviewed or verified. Please exercise caution in interpreting code and results, and consider validating all analyses and conclusions yourself.

---

# Overview

This notebook provides an introduction to **DANDI:001275**  
**Title:** Mental navigation primate PPC Neupane_Fiete_Jazayeri  
**Contributors:** Neupane, Sujaya  
**Institution:** MIT, Jazayeri and Fiete Labs  

This dataset contains neurophysiology recordings from two macaque monkeys performing a *mental navigation* task associated with this study: [Nature (2024)](https://doi.org/10.1038/s41586-024-07557-z).

It includes:
- Extracellular electrophysiology data
- Behavioral signals (eye, hand movement)
- Spike sorting results
- Metadata on sessions, electrodes, trials

**Note:** Data is streamed from the DANDI archive, so be mindful when loading large files or entire datasets.

## Accessing the Dandiset assets

Below we demonstrate how to use the DANDI API to list all assets/files in Dandiset `001275`:

In [1]:
from dandi.dandiapi import DandiAPIClient

client = DandiAPIClient()
dandiset = client.get_dandiset("001275", "draft")
assets = list(dandiset.get_assets())
print(f"Total assets in Dandiset 001275: {len(assets)}")
print("First 5 asset paths:")
for a in assets[:5]:
    print("-", a.path)

Total assets in Dandiset 001275: 20
First 5 asset paths:
- sub-mahler/sub-mahler_ses-03122021_behavior+ecephys.nwb
- sub-mahler/sub-mahler_ses-03172021_behavior+ecephys.nwb
- sub-mahler/sub-mahler_ses-03152021_behavior+ecephys.nwb
- sub-mahler/sub-mahler_ses-03182021_behavior+ecephys.nwb
- sub-mahler/sub-mahler_ses-03192021_behavior+ecephys.nwb


## Selecting NWB files to explore

This Dandiset contains multiple `.nwb` files, including recordings from different sessions and subjects.
Full list of filenames was shown above.

In this example, we will focus on one of the smaller files containing both behavioral and electrophysiology data:

`sub-amadeus/sub-amadeus_ses-01042020_behavior+ecephys.nwb`

## Loading NWB file via PyNWB + remfile

The cell below shows how to stream the NWB file remotely via DANDI without downloading the entire file locally:

In [2]:
import pynwb
import h5py
import remfile

nwb_url = "https://api.dandiarchive.org/api/assets/b0bbeb4c-5e0d-4050-a993-798173797d94/download/"
file = remfile.File(nwb_url)
f = h5py.File(file)
io = pynwb.NWBHDF5IO(file=f, load_namespaces=True)
nwbfile = io.read()

print("Session ID:", nwbfile.session_id)
print("Subject ID:", nwbfile.subject.subject_id if nwbfile.subject else "N/A")
print("Start time:", nwbfile.session_start_time)
if hasattr(nwbfile, 'lab'):
    print("Lab:", nwbfile.lab)
if hasattr(nwbfile, 'institution'):
    print("Institution:", nwbfile.institution)

/home/magland/miniconda3/envs/dev/lib/python3.12/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.6.0-alpha because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


Session ID: 01042020
Subject ID: amadeus
Start time: 2020-01-04 00:00:00-05:00
Lab: Jazayeri, Fiete
Institution: MIT


## NWB file content overview

Let's examine the key components of the NWB container, such as:
- Trials
- Units (spike times)
- Electrodes
- Processing modules (e.g., behavior, ecephys)

In [3]:
print("Available intervals (TimeIntervals):", list(nwbfile.intervals.keys()))
print("Available processing modules:", list(nwbfile.processing.keys()))

if "ecephys" in nwbfile.processing:
    ecephys_mod = nwbfile.processing["ecephys"]
    print("Ecephys module contents:", list(ecephys_mod.data_interfaces.keys()))

if "behavior" in nwbfile.processing:
    beh_mod = nwbfile.processing["behavior"]
    print("Behavior module contents:", list(beh_mod.data_interfaces.keys()))

Available intervals (TimeIntervals): ['trials']
Available processing modules: ['behavior', 'ecephys']
Ecephys module contents: ['units']
Behavior module contents: ['eye_position', 'hand_position']


## Inspect spikes and unit information

We can explore sorted units, their IDs, spike times, and (optionally) waveforms.

In [4]:
if "ecephys" in nwbfile.processing and "units" in nwbfile.processing["ecephys"].data_interfaces:
    units = nwbfile.processing["ecephys"].data_interfaces["units"]
    unit_ids = units.id[:]
    print(f"Total units: {len(unit_ids)}")
    print("First 10 unit IDs:", unit_ids[:10])

    # Example: spike times for first unit (if available)
    if len(unit_ids) > 0:
        spike_train = units["spike_times"][0][:]
        print(f"Spike train length of unit 0: {len(spike_train)}")
        print("First 10 spikes (s):", spike_train[:10])

Total units: 68
First 10 unit IDs: [0 1 2 3 4 5 6 7 8 9]


Spike train length of unit 0: 97283
First 10 spikes (s): [0.0065     1.21163333 1.24066667 1.24856667 1.34976667 1.3626
 1.37153333 1.3757     1.48413333 1.50966667]


## Trials data

Access the behavioral **trial events**:

In [5]:
if hasattr(nwbfile, 'trials') and nwbfile.trials is not None:
    print("Trial columns:", nwbfile.trials.colnames)
    print("Number of trials:", len(nwbfile.trials.id))
    print("First 5 trial IDs:", nwbfile.trials.id[:5])

Trial columns: ('start_time', 'stop_time', 'ta', 'tp', 'curr', 'target', 'rt', 'delay', 'trial_type', 'seqq', 'succ', 'attempt', 'numrepeat', 'validtrials_mm', 'gocuettl', 'joy1offttl', 'joy1onttl', 'stim1onttl')
Number of trials: 1885
First 5 trial IDs: [0 1 2 3 4]


## Accessing behavioral timeseries: eye and hand position

Behavior data may be found under the "behavior" processing module.

In [6]:
try:
    beh_mod = nwbfile.processing.get("behavior", None)
    if beh_mod:
        eye = beh_mod.data_interfaces.get("eye_position", None)
        hand = beh_mod.data_interfaces.get("hand_position", None)

        if eye:
            print("Eye position data shape:", eye.data.shape)
            print("Eye timestamps shape:", eye.timestamps.shape)
            print(f"Eye position units: {eye.unit}")

        if hand:
            print("Hand position data shape:", hand.data.shape)
            print("Hand timestamps shape:", hand.timestamps.shape)
            print(f"Hand position units: {hand.unit}")

except Exception as e:
    print(f"Error loading behavioral data: {e}")

Eye position data shape: (11891095, 2)
Eye timestamps shape: (11891095,)
Eye position units: meters
Hand position data shape: (11810979,)
Hand timestamps shape: (11810979,)
Hand position units: voltage


## Plotting examples (customize with your own data inspection)

Below code demonstrates *how* you might visualize behavioral data.
However, the explored sample file contained no continuous eye or hand data in the snippets.

Replace the indices or conditions as appropriate to your dataset.

In [7]:
import matplotlib.pyplot as plt

try:
    # Example: select snippet indices with data
    # eye_times = eye.timestamps[:]
    # eye_data = eye.data[:]
    # plt.plot(eye_times, eye_data[:,0], label='Eye X')
    # plt.plot(eye_times, eye_data[:,1], label='Eye Y')
    # plt.xlabel("Time (s)")
    # plt.ylabel("Eye position (meters)")
    # plt.legend()
    # plt.show()

    pass  # remove this and uncomment above as needed when data is available

except Exception as e:
    print(f"No behavioral snippet to plot: {e}")

## Summary and next steps

This notebook demonstrated how to:
- Connect to DANDI and list assets
- Stream NWB files remotely
- Access basic metadata: trials, units, electrodes
- Explore the data hierarchy interactively

**Next steps for researchers:**
- Select sessions/files of interest
- Explore spike data, LFP, behavioral traces in more depth
- Perform unit quality control, waveform analysis
- Cross-reference behavioral events and neural activity

**Reminder:** This notebook is AI-generated and intended as a scaffold. Please customize, validate, and extend as needed for your scientific analyses.